In [1]:
(define nil '())
(define (print x) (begin (display x) (newline)))

## Continuations

A continuation is a computation pending execution

> For example, 0 + 1 + 2 is expressed in Scheme as (+ 2 (+ 1 0)). From the perspective of 0, what will happen is that 1 will be added to it, and then 2 will be added to the result. The continuation of 0 in this context is therefore these two additions. [[Beautiful ideas in programming: generators and continuations](https://www.hhyu.org/posts/generator_and_continuation/)]

In [2]:
(define continuation nil)

(define store-continuation
  (lambda (cc) 
    (set! continuation cc)
    0))

(+ 2 (+ 1 (call/cc store-continuation)))  ; continuation is now (lambda (x) (+2 (+ 1 x)))
(print (continuation 1000))

1003

> In Scheme, we can use a function named `call/cc` to capture the current continuation, and then pass it to a different function, which is sometimes called the receiver. 

Consider the next Python function that uses an infinite generator:

In [7]:
(python-exec " 
def squared_ints():
    ''' Infinite generator, return i^2 for i = 1, 2, ... '''
    i = 1
    while True:
        yield i*i
        i += 1

def sum_of_squares_python(bound):
    ''' Return the smallest sum of squared integers greater than the bound '''
    s, g = 0, squared_ints()
    for i in g:      
        s += i       
        if s>bound:  
            return s
")

(sum_of_squares_python 100)  ; 1+4+9+16+25+36+49=140

140

Let's use `call/cc` to implement this Python function:

> Let’s implement the generator itself with call/cc. To make it happen, we need a mechanism to yield - to exit a function before it ends normally. We have already seen how it is done. Now, we need a mechanism that resumes the function after it has yielded. It’s not hard to see that we need two continuations.

In [8]:
(define squared-ints
  (lambda ()
    (let* ((break #f)                  ;;; will store a continuation to break out of the function
	       (resume #f)                 ;;; will store a continuation to resume after yielding
	       (yield                      ;;; define the function "yield"
	         (lambda (value)
	          (call/cc                 ;;;   capture the current continuation
	            (lambda (r)
		         (set! resume r)       ;;;   store it in "resume"
		         (break value))))))    ;;;   and break out 
      
      (lambda ()                       ;;; will return a function (a closure)
	    (call/cc                       ;;; capture the current continuation...
	      (lambda (cc)
	       (set! break cc)             ;;; ...and store it in "break"
	       (if resume                  ;;; if this generator has been called before...
	           (resume '())            ;;; ... resume it
	           (let loop ((i 1))       ;;; otherwise, loop through i=1, 2, 3, 4...
		         (yield (* i i))       ;;; yield the square of i
		         (loop (+ i 1))))))))))


(define sum-of-squares
  (lambda (bound)                        ;; define (sum-of-squares bound)
     (call/cc
       (lambda (break)                   ;; define a reciever: first capture the continuation into break
	     (let ((g (squared-ints)))       ;; make the generator g
	       (let loop ((s 0))             ;; define the recursive function (loop s), where s is the accumulated value
	         (let ((new-s (+ s (g))))    ;; compute the new accumulated value
	           (if (> new-s bound)       ;; if the new accumulated value is larger than bound
		           (break new-s)         ;; exit the loop
		           (loop new-s)))))))))  ;; otherwise, continue the loop with the new accumulated value

(sum-of-squares 100)

140

---

### Making a generator

The next code implements a generator-like object [[ref](https://jupyter.brynmawr.edu/services/public/dblank/CS245%20Programming%20Languages/2014-Fall/Notes/Call%20with%20Current%20Continuation.ipynb)]

In [16]:
(define generator
  (lambda (lst)
 
  ;; Hand the next item from a-list to "return" or an end-of-list marker
  (define control-state
   (lambda (yield)
    (for-each 
     (lambda (element)
               (set! yield (call/cc
                              (lambda (resume-here)
                                  ;; Grab the current continuation
                                  (set! control-state resume-here)
                                  (yield element)))))
     lst)
    (yield '())))
 
  ;; This is the actual generator, producing one item from a-list at a time
  (define start-generator
    (lambda ()
      (call/cc control-state)))
 
  ;; Return the generator 
  start-generator))
 
(define next
  (generator '(0 a 2)))

(print (next)) 
(print (next))
(print (next))
(print (next))
(print (next))

0
a
2
()
()


There is a module with generator procedures for Scheme at https://depp.brause.cc/srfi-158/

### Implementing a solver with continuations

In [17]:
; current-continuation : -> continuation
(define (current-continuation) 
  (call/cc 
   (lambda (cc)
     (cc cc))))

; fail-stack : list[continuation]
(define fail-stack '())

; fail : -> ...
(define (fail)
  (if (not (pair? fail-stack))
      (error fail "back-tracking stack exhausted!")
      (begin
        (let ((back-track-point (car fail-stack)))
          (set! fail-stack (cdr fail-stack))
          (back-track-point back-track-point)))))

; amb : list[a] -> a
(define (amb choices)
  (let ((cc (current-continuation)))
    (cond
      ((null? choices) (fail))
      ((pair? choices) (let ((choice (car choices)))
                         (set! choices (cdr choices))
                         (set! fail-stack (cons cc fail-stack))
                         choice)))))

; (assert condition) will cause
; condition to be true, and if there
; is no way to make it true, then
; it signals and error in the program.
(define (assert condition)
  (if (not condition)
      (fail)
      #t))

We're looking for dimensions of a legal right triangle using the Pythagorean theorem:

$a^2+b^2=c^2$

And, we want the second side (b) to be the shorter one:

$b<a$

In [25]:
(let ((a (amb (list 1 2 3   5 6 7 8 9 10)))
      (b (amb (list 1 2 3 4 5 6 7 8 9 10)))
      (c (amb (list 1 2 3 4 5 6 7 8 9 10))))
    
  (assert (= (* c c) (+ (* a a) (* b b))))
  (assert (< b a))

  (print (list a b c)))

(8 6 10)


Another reference: https://matt.might.net/articles/programming-with-continuations--exceptions-backtracking-search-threads-generators-coroutines/